In [1]:
#!pip install xmltodict

In [2]:
import pandas as pd
import time
import requests
import json
import xmltodict
import ast

In [3]:
base_url = 'https://boardgamegeek.com/xmlapi2/thing?'
args = '&ratingcomments=1&pagesize=100'
rating_args = '&stats=1'

In [4]:
def fetch_multiple_ratings(user_ratings, ids, page):
    name = ''
    ids = str.join(',', [str(x) for x in ids])
    has_more_pages = False
    url = f"{base_url}id={ids}{args}&page={page}"

    try:
        response = requests.get(url)
        # Parse XML response
        parsed = xmltodict.parse(response.content)
        
        items = parsed['items']['item']
        for item in items:
            id = item['@id']
            name = item['name'][0]['@value'] \
                if type(item['name']) is list else item['name']['@value']

            if page * 100 >= 100 + int(item['comments']['@totalitems']):
                continue
            comments = item['comments']['comment']

            if not isinstance(comments, list):
                comments = [comments]

            for j in range(0, len(comments)):
                user_ratings[id].append({
                    'id': id,
                    'name': name,
                    'user': comments[j]['@username'],
                    'rating': comments[j]['@rating']
                })
            # display(user_ratings)
            if page * 100 <= int(item['comments']['@totalitems']):
                has_more_pages = True
        return has_more_pages
    except:
        print(url)
        print(response.content)
        if comments:
            display(comments)
        raise Exception()

## Get ratings for a range of game ids, and save all to a CSV.

In [5]:
def fetch_multiple(ids):
    ratings = {}
    errors = []
    has_more = True
    page = 1
    for id in ids:
        ratings[f"{id}"] = []

    while has_more:
        try:
            has_more = fetch_multiple_ratings(ratings, ids, page)
            print(f"Page {page} complete.")
            page = page + 1
        except:
            failing = True
            while failing:
                time.sleep(5)
                try:
                    has_more = fetch_multiple_ratings(ratings, ids, page)
                    failing = False
                except:
                    err_str = f"Error getting page {page}."
                    errors.append(err_str)
                    print(err_str)

    dfs = []
    for id in ids:
        df = pd.DataFrame(ratings[f"{id}"])
        dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    print(errors)
    return df

def fetch_range(id_start, id_end):
    ids = [*range(id_start, id_end + 1)]
    df = fetch_multiple(ids)
    df.to_csv(f"game_{id_start}-{id_end}.csv")

## Filter games with a minimum user rating

In [6]:
def filter_on_rating(id_start, id_end, min_rating):
    ids = range(id_start, id_end)
    ids = str.join(',', [str(x) for x in ids])
    filtered_ids = []
    rating_counts = []
    try:
        response = requests.get(f"{base_url}id={ids}{rating_args}")
        # Parse XML response
        parsed = xmltodict.parse(response.content)
        
        items = parsed['items']['item']
        for item in items:
            id = item['@id']
            rating_count = int(item['statistics']['ratings']['usersrated']['@value'])
            rating = float(item['statistics']['ratings']['average']['@value'])
            if rating > min_rating:
                filtered_ids.append(int(id))
                rating_counts.append(rating_count)
            # display(item['statistics']['ratings']['average']['@value'])
    except:
        print(response.content)
        raise Exception()
    return [filtered_ids, rating_counts]
    
# ids = filter_on_rating(101, 200, 7)
# print(ids)
# print(len(ids))

In [12]:
id_start = 10001
id_end = 11000
min_rating = 7
ids, ratings = filter_on_rating(id_start, id_end, min_rating)
df = pd.DataFrame({'id': ids, 'rating_count': ratings})
df = df.loc[df['rating_count'] > 1].copy()
df

,id,rating_count
0,10013,24
1,10018,6
2,10019,13
3,10022,79
4,10055,7
...,...,...
90,10890,60
91,10904,96
92,10964,172
93,10971,36


In [8]:
df['id'].values

array([5024, 5042, 5045, 5058, 5069, 5072, 5087, 5117, 5169, 5171, 5205,
       5217, 5227, 5233, 5242, 5243, 5246, 5247, 5274, 5275, 5279, 5287,
       5288, 5289, 5290, 5291, 5292, 5293, 5305, 5324, 5351, 5357, 5369,
       5389, 5394, 5400, 5404, 5405, 5410, 5418, 5445, 5452, 5476, 5479],
      dtype=int64)

In [13]:
# df = fetch_multiple(ids)
df = fetch_multiple(df['id'].values)
# df = fetch_multiple([1803, 1818, 1821, 1822, 1872, 1879, 1883, 1887, 1890, 1894, 1915, 1929, 1966, 1967, 1968])
df.to_csv(f"game_{id_start}-{id_end}_min{min_rating}.csv")

Page 1 complete.
Page 2 complete.
Page 3 complete.
Page 4 complete.
Page 5 complete.
Page 6 complete.
Page 7 complete.
Page 8 complete.
Page 9 complete.
Page 10 complete.
Page 11 complete.
Page 12 complete.
Page 13 complete.
Page 14 complete.
Page 15 complete.
Page 16 complete.
Page 17 complete.
Page 18 complete.
Page 19 complete.
Page 20 complete.
Page 21 complete.
Page 22 complete.
Page 23 complete.
Page 24 complete.
Page 25 complete.
Page 26 complete.
Page 27 complete.
Page 28 complete.
Page 29 complete.
Page 30 complete.
Page 31 complete.
Page 32 complete.
Page 33 complete.
Page 34 complete.
Page 35 complete.
Page 36 complete.
Page 37 complete.
Page 38 complete.
Page 39 complete.
Page 40 complete.
Page 41 complete.
Page 42 complete.
Page 43 complete.
Page 44 complete.
Page 45 complete.
Page 46 complete.
Page 47 complete.
Page 48 complete.
Page 49 complete.
Page 50 complete.
Page 51 complete.
Page 52 complete.
Page 53 complete.
Page 54 complete.
Page 55 complete.
Page 56 complete.
P

In [18]:
# df1 = pd.read_csv('game_5001-5500_min7.csv')
# df2 = pd.read_csv('game_5501-6000_min7.csv')
# df3 = pd.read_csv('game_2701-3000_min7.csv')
df = pd.concat([
    pd.read_csv('game_5001-6000_min7.csv'),
    pd.read_csv('game_6001-6500_min7.csv'),
    pd.read_csv('game_6501-7000_min7.csv'),
    pd.read_csv('game_7001-7500_min7.csv'),
    pd.read_csv('game_7501-8000_min7.csv'),
    pd.read_csv('game_8001-8500_min7.csv'),
    pd.read_csv('game_8501-9000_min7.csv'),
    pd.read_csv('game_9001-9500_min7.csv'),
    pd.read_csv('game_9501-10000_min7.csv')
    # pd.read_csv('game_5501-6000_min7.csv'),
])
df = df[['id', 'name', 'user', 'rating']]
df.to_csv('game_5001-10000_min7.csv')

In [9]:
df_a = pd.read_csv('game_1.csv')
df_b = pd.read_csv('game_3.csv')
df_c = pd.read_csv('game_5.csv')
df_d = pd.read_csv('game_11.csv')
df_e = pd.read_csv('game_12.csv')
df_f = pd.read_csv('game_13.csv')
df_g = pd.read_csv('game_18.csv')
df = pd.concat([df_a, df_b, df_c, df_d, df_e, df_f, df_g])
df = df[['id', 'name', 'user', 'rating']]
df

df.to_csv(f"game_1-20_min7.csv")

In [27]:
df_a = pd.read_csv('game_1501-1800_min7.csv')
df_b = pd.read_csv('game_1801-2000_min7.csv')
# df_c = pd.read_csv('game_701-800_min7.csv')
# df_d = pd.read_csv('game_801-900_min7.csv')
# df_e = pd.read_csv('game_901-1000_min7.csv')
# df_f = pd.read_csv('game_401-500_min7.csv')

df = pd.concat([df_a, df_b])
# df = pd.concat([df_a, df_b, df_c, df_d, df_e])
df = df[['id', 'name', 'user', 'rating']]
df

df.to_csv(f"game_1501-2000_min7.csv")

In [13]:
df = df[['id', 'name', 'user', 'rating']]
df

,id,name,user,rating
0,42,Tigris & Euphrates,pjly,10.0
1,42,Tigris & Euphrates,gamegarage,10.0
2,42,Tigris & Euphrates,swiftlex,10.0
3,42,Tigris & Euphrates,martang,10.0
4,42,Tigris & Euphrates,HankM,10.0
...,...,...,...,...
103922,94,Union Pacific,jsect,1.0
103923,94,Union Pacific,Jumarik45,1.0
103924,94,Union Pacific,controlledinsanity,1.0
103925,94,Union Pacific,chrisandrachel,1.0
